In [ ]:
import gc
import torch
from transformers import AutoTokenizer

del llama_pipe

gc.collect()

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

In [2]:
from transformers import pipeline
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

llama_pipe = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16,
                "use_cache": True},
    device_map="auto"
)
print("Llama 3.1 pipeline loaded")


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.05it/s]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


Llama 3.1 pipeline loaded


In [ ]:
def chat(messages, max_new_tokens=2000):
    outputs = llama_pipe(messages, max_new_tokens=max_new_tokens)
    return outputs[0]["generated_text"]


In [4]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Now, please process the list:
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
c:\Users\hjt\Anaconda3\envs\LLMs-env\lib\site-packages\transformers\integrations\sdpa_attention.py:81: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nNow, please process the list:\n'}, {'role': 'user', 'content': "['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.']"}, {'role': 'assistant', 'content': "1. 'Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.' — habit"}]


# Zeroshot

In [5]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Now, please process the list:
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nNow, please process the list:\n'}, {'role': 'user', 'content': "['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcke

# Oneshot

In [6]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here is a structured one‑shot example to show how to answer:

Example:
'He checks his phone before going to bed every night.' — habit  

Now, please process the list:
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nHere is a structured one‑shot example to show how to answer:\n\nExample:\n\'He checks his phone before going to bed every night.\' — habit  \n\nNow, please process the list:\n'}, {'role': 'user', 'content': "['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negativ

# Fewshot

In [7]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here are our examples (two habits, two non‑habits):

Example 1:
'He checks his phone before going to bed every night.' — habit  

Example 2:
'I always read a book before going to sleep.' — habit  

Example 3:
'She visited Paris last summer.' — not a habit  

Example  4:
'I jog.' — not a habit    

Now, please process the list:
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nhabit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nHere are our exampl

# Definition

In [8]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Now, please process the list:
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nhabit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nNow, please process

# Definition and Oneshot

In [9]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here is a structured one‑shot example to show how to answer:

Example:
'He checks his phone before going to bed every night.' — habit  

Now, please process the list:
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nhabit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nHere is a structure

# Definition and Fewshot

In [10]:
messages = [
    {"role": "system", "content": """You are a habit recognition expert.

habit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.

Your task is to determine whether each sentence in a given list describes a habit.

The input is a list of 66 sentences, each enclosed in single quotation marks ('').  
Please evaluate the sentences in the exact order they appear in the list.

For each sentence, assign one of the following labels:  
- "habit" if the sentence describes a recurring or routine behavior,  
- "not a habit" if it does not.

The output should be a numbered list. Each item should include the sentence and your classification result in the following format:

[Number]. [Sentence] — [Label]

Here are our examples (two habits, two non‑habits):

Example 1:
'He checks his phone before going to bed every night.' — habit  

Example 2:
'I always read a book before going to sleep.' — habit  

Example 3:
'She visited Paris last summer.' — not a habit  

Example  4:
'I jog.' — not a habit  

Now, please process the list: 
"""},
    {"role": "user", "content": """['Morgens nach dem Aufstehen trinke ich direkt einen Kafffe.', 'Immer wenn ich müde bin, mache ich Atemübungen. ', 'Wenn ich ein negatives Gefühl stark fühle, versuche ich es auszufühlen. ', 'Abends nach dem Abendbrot trinke ich ein Glas Wein. ', 'Wenn ich am Wochenende zum Bäcker gehe, mache ich einen Abstecher in mein Lieblingscafe und trinke einen Cappucino. ', 'Nach dem Aufstehen mache ich Kraftsport', 'Sobald ich eine Aufgabe sehe, trage ich sie in Notion ein. ', 'Wenn mir in Teams angezeigt wird, dass sich ein Kollege im EvalGiGe Meeting befindet, dann trete ich dem Meeting bei. ', 'Wenn mich nach Feierabend noch niemand daheim erwartet, dann spaziere ich nach Hause. ', 'Einmal im Monat verbringe ich einen Tag mit mir allein in Dresden. ', 'Morgens nach dem Aufstehen wasche ich mein Gesicht.', 'Morgens/Abends nach dem Gesichtwaschen putze ich meine Zähne.', 'Morgens/Abends nach dem Zähneputzen creme ich mein Gesicht ein.', 'Morgens nach dem Gesicht-Eincremen füttere ich die Katze.', 'Morgens nach der Katzenfütterung bekommt der Hund eine Kaustange.', 'Morgens nach der Tierfütterung trinke ich einen Kaffee.', 'Nach dem Joggen trinke ich einen Proteinshake.', 'Wenn ich Lust auf ein Bier/Wein habe trinke ich ein Malzbier', 'Bevor ich schlafe mache ich meine Ohrstöpsel ins Ohr.', 'Bevor ich schlafen gehe nehme ich meine Nahrungsergänzungsmittel', 'Wenn ich gestresst bin, gehe ich zum Kühlschrank und nehme mir etwas zu essen.', 'Nach dem Aufstehen mache ich mir ein Zitronenwasser.', 'Wenn ich mich abends ins Bett lege, scrolle ich sinnlos durch social media.', 'wenn ich ins DM gehe, kaufe ich mir was Süßes.', 'nach dem Aufstehen trinken und Medikamenteinnahme mach ich ein kurzes HIT Workout', 'nach dem Abendbrot wasche ich ab', 'Nach dem Aufstehen mache ich ein paar Sprachübungen', 'Ich prüfe ob ich meinen Schlüssel dabeihabe, wenn ich aus der  WOhnung gehe', 'Ich nehme VItamin D tabletten ein', 'Samstags putze ich das Bad.', 'Wenn es nicht regnet, gieße ich im Sommer jeden Tag den Apfelbaum.', 'Nach dem Abendbrot an einem stressigen Tag esse ich gern ein Stück Schokolade', 'Nach dem Sport am Abend gehe ich oft erst spät ins Bett ', 'Ich habe mir heute eine neue Serie auf Netflix angesehen.', 'Ich war gestern im Kino', 'Ich habe vor, einen neuen Getränkegeschmack auszuprobieren', 'Ich plane, morgen ins Schwimmbad zu gehen.', 'Ich habe am Wochenende meine Freunde in einer anderen Stadt besucht.', 'Ich habe heute zum ersten Mal einen Podcast gehört.', 'Ich habe heute zum ersten Mal ein Selfie mit meiner neuen Kamera gemacht.', 'Heute habe ich einen langen Spaziergang durch eine fremde Gegend gemacht.', 'Ich habe letzte Woche meine Wohnung umdekoriert.', 'Letzte Woche war ich auf der Hochzeit meiner besten Freundin', 'Letzte Woche war ich auf der Beerdigung eines guten Freundes.', 'Ich habe vor, nächsten Mittwoch zum Arzt zu gehen', 'Mir geht es jetzt sehr schlecht, weil mein Magen weh tut', 'Ich bin an die Landschaft in den Bergen gewöhnt', 'Letzten Mittwoch hat es plötzlich geregnet, also bin ich in ein Café gegangen, das ich vorher nicht kannte, um Schutz vor dem Regen zu suchen, und ich habe festgestellt, dass es mir wirklich gut gefallen hat.', 'Heute Morgen fühlte ich mich plötzlich sehr müde', 'Haferbrei schmeckt besser, wenn man Blaubeeren dazu gibt', 'Es regnet heute, deshalb möchte ich nicht ins Fitnessstudio gehen', 'Ich habe mir den Rap einmal angehört und er klang ziemlich gut.', 'Heute habe ich versucht zu lernen, wie man einen Kuchen backt', 'Ich habe heute viele neue Freunde kennengelernt', 'Ich habe heute viele neue Möbel zur Einrichtung meines neuen Hauses bestellt.', 'Wenn ich versuche zu bezahlen, friert die Webseite aus unbekannten Gründen immer wieder ein', 'Vielleicht plane ich, in Zukunft zum Militär zu gehen.', 'Ich habe gestern einen Film auf Englisch ohne Untertitel gesehen.', 'Ich habe mir heute eine neue Pflanze gekauft.', 'Ich jogge', 'Letzte Woche war ich joggen.', 'Ich habe einmal Tischtennis gespielt und es nie wieder angefasst', 'Ich liebe Pommes Frites', 'Ich habe gestern Pommes gegessen', 'Ich mag Eis, aber nicht jedes Mal.', 'Gestern habe ich zum ersten Mal Yoga gemacht.']"""}
]

response = chat(messages)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'role': 'system', 'content': 'You are a habit recognition expert.\n\nhabit is a process by which a stimulus generates an impulse to act as a result of a learned stimulus-response association. Habit-generated impulses may compete or combine with impulses and inhibitions arising from other sources, including conscious decisionmaking, to influence responses, and need not generate behaviour.\n\nYour task is to determine whether each sentence in a given list describes a habit.\n\nThe input is a list of 66 sentences, each enclosed in single quotation marks (\'\').  \nPlease evaluate the sentences in the exact order they appear in the list.\n\nFor each sentence, assign one of the following labels:  \n- "habit" if the sentence describes a recurring or routine behavior,  \n- "not a habit" if it does not.\n\nThe output should be a numbered list. Each item should include the sentence and your classification result in the following format:\n\n[Number]. [Sentence] — [Label]\n\nHere are our exampl